In [ ]:
import numpy as np
import random
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]  # select best two
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]

for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# Best result
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("\nBest Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])

# ============================================================================
# GENETIC ALGORITHM FOR MACHINE LEARNING HYPERPARAMETER OPTIMIZATION
# ============================================================================
#
# GOAL: Find the best hyperparameters for a Decision Tree to maximize accuracy
#
# APPROACH: Use Genetic Algorithm (inspired by natural evolution)
# - Population: 20 candidate solutions (sets of hyperparameters)
# - Evolution: 10 generations of selection, crossover, and mutation
# - Fitness: Cross-validation accuracy (higher = better)
#
# KEY CONCEPT - GENETIC ALGORITHM:
# 1. Start with random solutions (population)
# 2. Evaluate each solution (fitness)
# 3. Select best solutions (parents)
# 4. Combine parents to create offspring (crossover)
# 5. Randomly modify offspring (mutation)
# 6. Repeat for multiple generations
# 7. Best solution emerges through evolution
#
# ANALOGY: Like breeding animals - keep best traits, combine them,
#          occasionally introduce random changes
#
# ============================================================================
# IMPORTS
# ============================================================================
#
# import numpy as np
# - For numerical operations and array handling
#
# import random
# - For random number generation in GA operations
#
# from sklearn.datasets import load_iris
# - Load famous Iris flower dataset (150 samples, 3 classes)
#
# from sklearn.model_selection import cross_val_score
# - Evaluate model using k-fold cross-validation (prevents overfitting)
#
# from sklearn.tree import DecisionTreeClassifier
# - Machine learning model we're optimizing
#
# ============================================================================
# LOAD DATASET
# ============================================================================
#
# iris = load_iris()
# - Loads Iris dataset containing:
#   * 150 samples (flowers)
#   * 4 features (sepal length, sepal width, petal length, petal width)
#   * 3 classes (Setosa, Versicolor, Virginica)
#
# X, y = iris.data, iris.target
# - X: Feature matrix (150 samples × 4 features)
# - y: Target labels (150 labels: 0, 1, or 2)
#
# ============================================================================
# GA PARAMETERS
# ============================================================================
#
# POP_SIZE = 20
# - Number of candidate solutions in each generation
# - More = better exploration but slower
#
# N_GENERATIONS = 10
# - Number of evolution cycles
# - More = better convergence but slower
#
# MUTATION_RATE = 0.2
# - Probability of random change (20% chance)
# - Prevents getting stuck in local optima
#
# ============================================================================
# CHROMOSOME REPRESENTATION
# ============================================================================
#
# Each chromosome encodes 2 hyperparameters as a list:
# [max_depth, min_samples_split]
#
# max_depth (1-20):
# - Maximum depth of decision tree
# - Deeper = more complex, may overfit
# - Shallower = simpler, may underfit
#
# min_samples_split (2-10):
# - Minimum samples required to split a node
# - Higher = simpler tree (fewer splits)
# - Lower = more complex tree (more splits)
#
# EXAMPLE: [8, 5] means max_depth=8, min_samples_split=5
#
# ============================================================================
# CREATE CHROMOSOME
# ============================================================================
#
# def create_chromosome():
#     return [random.randint(1, 20), random.randint(2, 10)]
#
# - Generates random hyperparameter values
# - max_depth: random integer between 1 and 20
# - min_samples_split: random integer between 2 and 10
# - Returns list with 2 values
#
# EXAMPLE OUTPUT: [12, 7] or [3, 9] or [18, 4]
#
# ============================================================================
# FITNESS FUNCTION
# ============================================================================
#
# def fitness(chromosome):
#     max_depth, min_samples_split = chromosome
#
# - Unpacks chromosome into individual hyperparameters
# - Example: [8, 5] → max_depth=8, min_samples_split=5
#
#     model = DecisionTreeClassifier(max_depth=max_depth,
#                                    min_samples_split=min_samples_split)
#
# - Creates Decision Tree with specified hyperparameters
# - These parameters control tree complexity and generalization
#
#     scores = cross_val_score(model, X, y, cv=5)
#
# - Evaluates model using 5-fold cross-validation
# - Splits data into 5 parts, trains on 4, tests on 1, repeats 5 times
# - Returns array of 5 accuracy scores
# - Example: [0.93, 0.97, 0.90, 0.93, 0.97]
#
# WHY CROSS-VALIDATION?
# - Prevents overfitting (testing on unseen data)
# - More reliable than single train-test split
# - Uses all data for both training and testing
#
#     return scores.mean()
#
# - Returns average accuracy across 5 folds
# - Example: (0.93 + 0.97 + 0.90 + 0.93 + 0.97) / 5 = 0.94
# - This is the fitness value (higher = better)
#
# ============================================================================
# SELECTION
# ============================================================================
#
# def selection(population, fitnesses):
#     idx = np.argsort(fitnesses)[-2:]
#
# - np.argsort(fitnesses): Returns indices that would sort array
#   Example: fitnesses = [0.85, 0.92, 0.78, 0.95, 0.88]
#            argsort gives: [2, 0, 4, 1, 3] (sorted order)
#
# - [-2:]: Takes last 2 indices (highest fitness)
#   Example: [1, 3] (individuals at positions 1 and 3 are best)
#
#     return [population[idx[0]], population[idx[1]]]
#
# - Returns the 2 best chromosomes as parents
# - These will be used to create offspring
#
# SELECTION STRATEGY: Elitism (always keep best)
# - Ensures good solutions aren't lost
# - Simple and effective
#
# ============================================================================
# CROSSOVER
# ============================================================================
#
# def crossover(parent1, parent2):
#     point = random.randint(0, len(parent1)-1)
#
# - Chooses random crossover point (0 or 1)
# - Determines where to split chromosomes
# - Example: point = 1
#
#     child1 = parent1[:point] + parent2[point:]
#     child2 = parent2[:point] + parent1[point:]
#
# - Combines parts of both parents
#
# EXAMPLE:
# parent1 = [15, 3]  parent2 = [8, 7]  point = 1
# child1 = [15] + [7] = [15, 7]  (max_depth from p1, split from p2)
# child2 = [8] + [3] = [8, 3]    (max_depth from p2, split from p1)
#
# PURPOSE: Combine good traits from both parents
#
#     return child1, child2
#
# ============================================================================
# MUTATION
# ============================================================================
#
# def mutate(chromosome):
#     if random.random() < MUTATION_RATE:
#         chromosome[0] = random.randint(1, 20)
#
# - 20% chance to randomly change max_depth
# - Introduces new genetic material
# - Helps explore new solutions
#
#     if random.random() < MUTATION_RATE:
#         chromosome[1] = random.randint(2, 10)
#
# - 20% chance to randomly change min_samples_split
# - Independent from first mutation
#
#     return chromosome
#
# EXAMPLE:
# Input: [15, 7]
# If both mutations happen: [12, 4]
# If only first: [18, 7]
# If neither: [15, 7]
#
# PURPOSE: Prevent premature convergence, explore new regions
#
# ============================================================================
# INITIALIZE POPULATION
# ============================================================================
#
# population = [create_chromosome() for _ in range(POP_SIZE)]
#
# - Creates 20 random chromosomes
# - List comprehension calls create_chromosome() 20 times
#
# EXAMPLE:
# [[15, 3], [8, 7], [12, 5], [3, 9], ..., [18, 4]]
# 20 different random hyperparameter combinations
#
# ============================================================================
# MAIN GA LOOP
# ============================================================================
#
# for gen in range(N_GENERATIONS):
# - Runs for 10 generations (evolution cycles)
#
#     fitnesses = [fitness(chromo) for chromo in population]
#
# - Evaluates all 20 chromosomes
# - Each fitness call trains and tests a Decision Tree
# - Returns list of 20 accuracy values
# - Example: [0.85, 0.92, 0.78, 0.95, ..., 0.88]
#
#     print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")
#
# - Displays best accuracy in current generation
# - Shows optimization progress
# - Example: "Generation 0 - Best Fitness: 0.9533"
#
#     new_population = []
# - Empty list to store offspring
#
#     parents = selection(population, fitnesses)
#
# - Selects 2 best chromosomes from current population
# - These will be parents for all offspring
# - Example: [[15, 3], [8, 7]]
#
#     for _ in range(POP_SIZE // 2):
#
# - Loop 10 times (20 // 2 = 10)
# - Each iteration creates 2 children (total 20)
#
#         child1, child2 = crossover(parents[0], parents[1])
#
# - Combines parents to create 2 offspring
# - Offspring inherit traits from both parents
#
#         new_population.append(mutate(child1))
#         new_population.append(mutate(child2))
#
# - Applies mutation to each child
# - Adds both to new population
# - After 10 loops: 20 new individuals
#
#     population = new_population
#
# - Replaces old generation with new generation
# - Evolution continues with improved population
#
# ============================================================================
# FINAL RESULT
# ============================================================================
#
# fitnesses = [fitness(chromo) for chromo in population]
# - Evaluates final generation
#
# best_idx = np.argmax(fitnesses)
# - Finds index of best chromosome
# - argmax returns position of maximum value
#
# print("\nBest Hyperparameters:", population[best_idx])
# - Displays optimal hyperparameters found
# - Example: [8, 5]
#
# print("Best Accuracy:", fitnesses[best_idx])
# - Displays best accuracy achieved
# - Example: 0.9667
#
# ============================================================================
# COMPLETE EXAMPLE RUN
# ============================================================================
#
# Generation 0: Random population, best = 0.9267
# Generation 1: After evolution, best = 0.9467
# Generation 2: Improving, best = 0.9533
# Generation 5: Converging, best = 0.9600
# Generation 9: Final, best = 0.9667
#
# Best Hyperparameters: [8, 5]
# Best Accuracy: 0.9667
#
# INTERPRETATION:
# - max_depth=8: Moderately deep tree
# - min_samples_split=5: Moderate splitting threshold
# - Accuracy=96.67%: Very good performance
#
# ============================================================================
# WHY GA WORKS
# ============================================================================
#
# 1. EXPLORATION: Random initial population covers search space
# 2. EXPLOITATION: Selection keeps best solutions
# 3. RECOMBINATION: Crossover combines good traits
# 4. INNOVATION: Mutation introduces new possibilities
# 5. ITERATION: Process repeats, gradually improving
#
# ADVANTAGE OVER MANUAL TUNING:
# - Automatic (no human trial-and-error)
# - Explores many combinations efficiently
# - Finds near-optimal solutions
# - Better than random search or grid search
#
# ============================================================================
# KEY TAKEAWAYS
# ============================================================================
#
# - GA mimics natural evolution to find optimal hyperparameters
# - Population evolves over generations through selection and reproduction
# - Fitness function guides evolution toward better solutions
# - Works well for hyperparameter optimization
# - Simple to implement, effective for many problems
#
# ============================================================================
# END OF EXPLANATION
# ============================================================================

Generation 0 - Best Fitness: 0.9667
Generation 1 - Best Fitness: 0.9667
Generation 2 - Best Fitness: 0.9667
Generation 3 - Best Fitness: 0.9667
Generation 4 - Best Fitness: 0.9667
Generation 5 - Best Fitness: 0.9667
Generation 6 - Best Fitness: 0.9667
Generation 7 - Best Fitness: 0.9667
Generation 8 - Best Fitness: 0.9667
Generation 9 - Best Fitness: 0.9667

Best Hyperparameters: [8, 9]
Best Accuracy: 0.9666666666666668
